# 13.7 消耗Token记录

记录每次请求使用Token数量。

场景：
- 商业产品：WPS,购买了ZhipuAI的推理服务，会员可以使用。（多人共用一个API_KEY）
- 企业内部，使用外部的LLM服务，根据Token记录每个人的花费（多人共用一个API_KEY）

### Decimal
用来进行精确的十进制计算。
- quantize(exp, rounding=None, context=None)

银行家舍入法：

1. “四”是指≤4时舍去，"六"是指≥6时进上
2. 当5后有数时，舍5入1
3. 当5后无有效数字时，5前为奇数，舍5入1
4. 当5后无有效数字时，5前为偶数，舍5不进（0是偶数）。

```python
9.8249=9.82
9.82671=9.83
9.835=9.84
9.8351 =9.84
9.825=9.82
9.82501=9.83
```

In [50]:
"""
gpt-4o：
提问：$2.50 / 1M input tokens -> 0.0025 $ / 1000 token -> 0.0025 * 7.2 = 0.018¥
回答：$10.00 / 1M output tokens
Pricing 批量API：
提问：$1.25 / 1M input tokens
回答：$5.00 / 1M output tokens
"""
from decimal import Decimal, ROUND_HALF_EVEN
price_info = {
    "glm-4": {
        "prompt_price": Decimal(0.1), # 1000 token
        "completion_price": Decimal(0.1),
    }
}
# llm_output={'token_usage': {'completion_tokens': 554, 'prompt_tokens': 15, 'total_tokens': 569}, 'model_name': 'glm-4'}
def compute_cost(llm_output):
    if not llm_output:
        raise Exception("llm_output为空")
    token_usage = llm_output.get("token_usage", None)
    if not token_usage:
        raise Exception("token_usage为空")
    prompt_tokens = token_usage.get("prompt_tokens", Decimal(0))
    completion_tokens = token_usage.get("completion_tokens", Decimal(0))

    model_name = llm_output.get("model_name", "glm-4")

    _price_info = price_info.get(model_name)
    prompt_price = _price_info.get("prompt_price", Decimal(0))
    completion_price = _price_info.get("completion_price", Decimal(0))

    prompt_cost = Decimal(prompt_tokens) / Decimal(1000) * prompt_price
    completion_cost = Decimal(completion_tokens) / Decimal(1000) * completion_price

    cost = prompt_cost + completion_cost
    cost = cost.quantize(Decimal("0.0001"),rounding=ROUND_HALF_EVEN)
    return {
        "cost": cost,
        "model_name": model_name,
        "token_usage": token_usage,
    }
    

In [51]:
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages import BaseMessage
from langchain_core.outputs import ChatGenerationChunk, GenerationChunk, LLMResult
from typing import TYPE_CHECKING, Any, Dict, List, Optional, Sequence, TypeVar, Union
from uuid import UUID

# LCEL: langchain expression language(langchain表达式语言)
class LCELEventHandler(BaseCallbackHandler):  
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"on_llm_end,response: {response}, kwargs:{kwargs}")
        cost = compute_cost(response.llm_output)
        print(f"cost:{cost}")


In [52]:
# ChatPromptTempalte
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama, ChatZhipuAI
from langchain_core.output_parsers import StrOutputParser
import os
import getpass

os.environ["ZHIPUAI_API_KEY"] =  getpass.getpass("请输入ZHIPUAI_API_KEY...")

prompt = ChatPromptTemplate.from_template("请你使用中文描述一下LangChain的事件体系？")


# 1. 创建llm对象
# chat_model = ChatOllama(model="llama3")
chat_model = ChatZhipuAI(model="glm-4")

# 2. 创建一个langchain对象
parser = StrOutputParser()
chain = (prompt.with_config({"run_name":"prompt_dxh"}) 
| chat_model.with_config({"run_name":"chat_model_dxh"}) 
| parser.with_config({"run_name":"parser_dxh"}) )

chain = chain.with_config({"callbacks": [LCELEventHandler()], "run_name": "chain_dxh"})

# 1. invoke()调用
# result = chain.invoke({},{"callbacks": [LCELEventHandler()]})
# 2. ainvoke()调用
# result = await chain.ainvoke({})
# 3. stream()调用
# result = chain.stream({})
# for chunk in result:
#     ...
# 4. astream()调用
result = chain.astream({})
async for chunk in result:
    ...

请输入ZHIPUAI_API_KEY... ········


Error in LCELEventHandler.on_llm_end callback: Exception('llm_output为空')


on_llm_end,response: generations=[[ChatGenerationChunk(text='LangChain是一个基于语言模型的人工智能框架，其事件体系主要是为了处理和响应在链中传递的各种事件。尽管“LangChain”一词在不同的上下文中可能有不同的含义，但一般来说，我们可以将事件体系描述为以下特点：\n\n1. **事件定义**：\n   - 在LangChain中，事件可以理解为一系列预定义的动作或消息，这些动作或消息在链中的各个节点之间传递，触发相应的处理逻辑。\n\n2. **事件类型**：\n   - **输入事件**：这是用户或其他系统发送到LangChain的初始请求或指令，它将启动整个链的运行。\n   - **处理事件**：在链中流动的处理过程中触发的事件，可能包括数据处理、状态更新等。\n   - **输出事件**：这是LangChain处理完毕后生成的结果或反馈，通常包括对用户请求的响应。\n\n3. **事件处理**：\n   - LangChain的事件体系设计为能够处理各种类型的输入，并将其转化为有用的输出。\n   - **节点处理**：在LangChain中，不同的节点可能负责处理不同类型的事件。每个节点在接收到事件后，根据预设的逻辑执行相关的操作。\n\n4. **事件流**：\n   - 事件在LangChain中的流动形成了一个序列，每个事件都可能依赖于前一个事件的输出。\n   - 这个序列通常是线性的，但也可以设计为有条件分支的结构，以处理复杂的逻辑。\n\n5. **错误处理**：\n   - 在事件体系中，错误处理是一个关键组成部分。当事件处理失败时，LangChain应该有能力捕捉异常，并采取相应的措施，如重试、回退或报告错误。\n\n6. **状态管理**：\n   - LangChain需要维护一个状态管理机制来跟踪事件处理的状态，确保信息的一致性和上下文的连续性。\n\n7. **集成和扩展**：\n   - 事件体系还应该支持与外部系统集成，允许将LangChain的能力与其他系统或服务进行整合。\n   - 同时，为了适应未来的需求，事件体系应具备扩展性，允许引入新的类型的事件和处理器。\n\n总的来说，LangChain的事件体系是为了保证信息的流动和处理的灵活性、可靠性和